In [1]:
site_name = "Sturt Plains"
site_latitude = -17.15090
site_longitude = 133.35055

In [2]:
; define the DAP URL and the file name
dap_path = "http://www.auscover.org.au/thredds/dodsC/auscover/lpdaac-aggregates/c5/v2-nc4/aust/"
evi_file = "MOD13Q1.005/MOD13Q1.aggregated.aust.005.enhanced_vegetation_index.ncml"

In [3]:
; get the full URL and open the netCDF file
evi_full_name = dap_path+evi_file
evi_file = ncdf_open(evi_full_name)

In [4]:
; get the latitude and longitude resolutions from the global attributes
ncdf_attget,evi_file,"geospatial_lat_resolution",lat_res,/global
ncdf_attget,evi_file,"geospatial_lon_resolution",lon_res,/global

In [5]:
; get the time variable
time_id = ncdf_varid(evi_file,"time")
ncdf_varget,evi_file,time_id,time
nrecs = n_elements(time)
; and the time variable units attribute
ncdf_attget,evi_file,time_id,"units",value
time_units = string(value)
; get the year, month and day from the time units string
result = strsplit(time_units," ",/extract)
ymd = strsplit(result[2],"-",/extract)
; get the hour, minute and seconds
hms = strsplit(result[3],":",/extract)
; get the Julian date from the netCDF time
jul_time = time + JulDay(fix(ymd[2]),fix(ymd[1]),fix(ymd[0]),0,0,0)
; ... and get the year, month, day etc from the Julian date
CalDat, jul_time, month, day, year, hour, min, sec
;print, year[-1], month[-1], day[-1], hour[-1], min[-1], sec[-1]

In [6]:
; get the latitude and longitude variables from the netCDF file
lat_id = ncdf_varid(evi_file,"latitude")
ncdf_varget,evi_file,lat_id,latitude
lon_id = ncdf_varid(evi_file,"longitude")
ncdf_varget,evi_file,lon_id,longitude

In [7]:
; NOTE: evi is dimensioned as [longitude,latitude,time]
;                          eg [19160,14902,365]
; get the latitude and longitude indices
lat_index = fix(((latitude[0]-site_latitude)/lat_res)+0.5)
; if site_longitude<0: site_longitude = float(360) + site_longitude
lon_index = fix(((site_longitude-longitude[0])/lon_res)+0.5)
; get the offset and count for the data subset
offset = [lon_index-1,lat_index-1,0]
count = [3,3,nrecs]
 ; and now get the EVI data
evi_id = ncdf_varid(evi_file,"evi")
ncdf_varget,evi_file,evi_id,evi,offset=offset,count=count

In [8]:
!inline=1
date_label = label_date(DATE_FORMAT = ['%Y/%M'])
window,0
plot,jul_time,evi[1,1,*],max_value=1.0,min_value=-0.2,XTICKFORMAT = 'LABEL_DATE'

% Warning: Assignment to loop variable detected.
% Warning: Assignment to loop variable detected.
